# Optimization without precession (Replaces by Precession)

In [ ]:
def main_original(): 

    n_workers = 5

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    mass1_template = [60*lal.MSUN_SI, 60*lal.MSUN_SI, 60*lal.MSUN_SI, 20*lal.MSUN_SI, 20*lal.MSUN_SI]
    mass2_template = [20*lal.MSUN_SI, 20*lal.MSUN_SI, 20*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI]
    eff_spin_template = [0.0, -0.7, 0.7, -0.5, 0.5]

    prms_initial = []
    for i in range(n_workers):
        mass_ratio_template = mass1_template[i]/mass2_template[i]
        chirp_mass_template = chirp_mass_function([mass1_template[i], mass2_template[i]])
        prms_initial.append([mass_ratio_template, chirp_mass_template, eff_spin_template[i]])

    print("Starting Hierarchical Optimization")
    print("First optimization: Masses and Effective Spin")
    time_initial=time.time()

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_original_first)(prms) for prms in prms_initial)
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
    
    print(f"The match of the first optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(F"The first optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2]}.") # Print the parameters calculated by the global optimization
        
    spin2z_template = [0.0, -0.9, -0.5, 0.5, 0.9]
    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], spin2z_template[i]])

    print(f"Second Optimization: Spins")

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_original_second)(prms) for prms in prms_initial)
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]

    masses_final = M_c_and_q_m(prms_final[0], prms_final[1])
    spins_final = Eff_spin_and_spin1(masses_final[0], masses_final[1], prms_final[2], prms_final[3])

    print(f"The match of the total optimization is: {max_match}. The complete optimization took {time.time()-time_initial} seconds.")

    print(f"Optimum at masses {masses_final[0]/lal.MSUN_SI} and {masses_final[1]/lal.MSUN_SI} solar masses and spins {spins_final[0]} and {spins_final[1]}")
    print(f"The target: {parameters_target.m1/lal.MSUN_SI} and {parameters_target.m2/lal.MSUN_SI} solar masses and spins {parameters_target.s1z} and {parameters_target.s2z}")
    print(f"chirp mass: {prms_final[1]/lal.MSUN_SI}, chirp mass target: {parameters_target.chirp_mass()/lal.MSUN_SI}")
    print(f"effective spin parameter: {prms_final[2]}, eff_spin_target: {parameters_target.eff_spin()}")

    # Waveforms Coaligned 
    hp_test, hc_test, time_test = simulationTD_IMRPhenomTPHM(params(masses_final, (0,0,spins_final[0]), (0,0,spins_final[1])))
    h1_aligned, h2_aligned = coalign_waveforms(hp_target, TimeSeries(hp_test, delta_t=delta_T))

    # Plot of the coaligned waveforms
    plt.figure(figsize=(12, 5))
    plt.plot(h1_aligned.sample_times, h1_aligned, label = f'Target')
    plt.plot(h2_aligned.sample_times, h2_aligned, label = f'Template.', linestyle='dashed')
    plt.title(f'The match between the gravitational waves is {max_match}')
    plt.xlabel('Time (s)')
    plt.ylabel('Strain')
    plt.legend()

    plt.savefig('./Graphics/main2.png', bbox_inches='tight') # Guardar la imagen como un png
    plt.show() # Mostrar la imagen

    return 0

# Global Initial Optimization (Replaced by Hierarchical Algorithm) 

In [ ]:
def opt_precession_global_high_Q(prms_initial:list)->tuple:

    opt = nlopt.opt(nlopt.GN_MLSL, 6) # Define the local optimization
    opt.set_lower_bounds([1, lal.MSUN_SI, -1, -1, -1, -1]) # Same boundaries as before
    opt.set_upper_bounds([20, 175*lal.MSUN_SI, 1, 1, 1, 1]) # Same boundaries as before
    opt.set_min_objective(opt_match_precession_Big_Q)
    opt.set_maxeval(200)

    prms_final = opt.optimize(prms_initial) # We use the parameters obtained by the global optimization as the starting point
    max_match = -opt.last_optimum_value()

    return max_match, prms_final


def Global_Borrar():

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    mass1_template = 60*lal.MSUN_SI
    mass2_template = 20*lal.MSUN_SI
    eff_spin_template = 0.0
    spin2z_template = 0.0
    spin1x_template = 0.0
    spin1y_template = 0.0

    prms_initial = [mass1_template/mass2_template, chirp_mass_function([mass1_template, mass2_template]), eff_spin_template, spin2z_template,
                    spin1x_template, spin1y_template]


    print("Starting Global Optimization")
    print("First optimization: Global")
    time_initial=time.time()
    max_match, prms_final = opt_precession_global_high_Q(prms_initial)

    
    print(f"The match of the first optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(F"The first optimization has obtain the following parameters: {prms_final}.") # Print the parameters calculated by the global optimization
    
    
    prms_initial = prms_final

    print(f"Second Optimization: Local")

    max_match, prms_final = opt_precession_high_Q(prms_initial)

    masses_final = M_c_and_q_m(prms_final[0], prms_final[1])
    spin1z_final, spin2z_final = Eff_spin_and_spin1(masses_final[0], masses_final[1], prms_final[2], prms_final[3])

    print(f"The match of the total optimization is: {max_match}. The complete optimization took {time.time()-time_initial} seconds.")

    print(f"Optimum at masses {masses_final[0]/lal.MSUN_SI} and {masses_final[1]/lal.MSUN_SI} solar masses and spins {prms_final[4], prms_final[5], spin1z_final} and {spin2z_final}")
    print(f"The target: {parameters_target.m1/lal.MSUN_SI} and {parameters_target.m2/lal.MSUN_SI} solar masses and spins {parameters_target.s1x, parameters_target.s1y, parameters_target.s1z} and {parameters_target.s2z}")
    print(f"chirp mass: {prms_final[1]/lal.MSUN_SI}, chirp mass target: {parameters_target.chirp_mass()/lal.MSUN_SI}")
    print(f"effective spin parameter: {prms_final[2]}, eff_spin_target: {parameters_target.eff_spin()}")

    # Waveforms Coaligned 
    hp_test, hc_test, time_test = simulationTD_IMRPhenomTPHM(params(masses_final, (prms_final[4],prms_final[5],spin1z_final), (0,0,spin2z_final)))
    h1_aligned, h2_aligned = coalign_waveforms(hp_target, TimeSeries(hp_test, delta_t=delta_T))

    # Plot of the coaligned waveforms
    plt.figure(figsize=(12, 5))
    plt.plot(h1_aligned.sample_times, h1_aligned, label = f'Target')
    plt.plot(h2_aligned.sample_times, h2_aligned, label = f'Template.', linestyle='dashed')
    plt.title(f'The mismatch between the gravitational waves is {1-max_match}')
    plt.xlabel('Time (s)')
    plt.ylabel('Strain')
    plt.legend()

    plt.savefig('./Graphics/main2.png', bbox_inches='tight') # Guardar la imagen como un png
    plt.show() # Mostrar la imagen

    return 0

# Hierarchical Code with 3 steps without inclination (Replaced by 2 steps)

In [ ]:
def main_not_Inclination(a): 

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    n_workers = 6

    # We write the initial values for the optimization

    mass1_template = [8*lal.MSUN_SI, 8*lal.MSUN_SI, 100*lal.MSUN_SI, 100*lal.MSUN_SI, 40*lal.MSUN_SI, 40*lal.MSUN_SI]
    mass2_template = [4*lal.MSUN_SI, 4*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI]
    eff_spin_template = [-0.5, 0.5, 0.7, -0.7, 0.7, -0,7]


    prms_initial = []
    for i in range(n_workers):
        mass_ratio_template = mass1_template[i]/mass2_template[i]
        chirp_mass_template = chirp_mass_function([mass1_template[i], mass2_template[i]])
        prms_initial.append([mass_ratio_template, chirp_mass_template, eff_spin_template[i]])

    print("Starting Hierarchical Optimization")
    print("First optimization: Masses and Effective Spin")
    time_initial=time.time()

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_first)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")
    
    # max_match, prms_final = opt_first(prms_initial[3])
    
    print(colored(f"The match of the first optimization is: {max_match}. It took {time.time()-time_initial} seconds.", "magenta"))
    print(f"The first optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2]}.") # Print the parameters calculated by the global optimization

    print(f"Second Optimization: Spin2z and Inclination")

    n_workers = 3
    spin2z_template = [0.0, -0.5, 0.5]

    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], spin2z_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_second_not_incl)(prms) for prms in prms_initial)

    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker][3]}")


    print(colored(f"The match of the second optimization is: {max_match}. It took {time.time()-time_initial} seconds.", "magenta"))
    print(f"The second optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2], prms_final[3]}.") # Print the parameters calculated by the global optimization

    print(f"Third Optimization: Spin1x and Spin1y")

    n_workers = 5
    spin1x_template = [0.0, 0.4, -0.4, -0.4, 0.4]
    spin1y_template = [0.0, 0.4, -0.4, 0.4, -0.4]
    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], prms_final[3], spin1x_template[i], spin1y_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_third_precession_not_incl)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker][4], prms_initial[best_worker][5]}")

    # max_match, prms_final = opt_third_precession(prms_initial[2])

    masses_final = func.M_c_and_q_m(prms_final[0], prms_final[1])
    spin1z_final, spin2z_final = func.Eff_spin_and_spin1(masses_final[0], masses_final[1], prms_final[2], prms_final[3])

    print(colored(f"The match of the total optimization is: {max_match}. The complete optimization took {time.time()-time_initial} seconds.", "magenta"))
    
    print(f"Parameters: {prms_final}")
    print(f"chirp mass: {prms_final[1]/lal.MSUN_SI}, chirp mass target: {parameters_target.chirp_mass()/lal.MSUN_SI}")
    print(f"effective spin parameter: {prms_final[2]}, eff_spin_target: {parameters_target.eff_spin()}")

    # Waveforms Coaligned 
    hp_test, hc_test, time_test = simulationTD("IMRPhenomTPHM", params(masses_final, (prms_final[4],prms_final[5],spin1z_final), (0,0,spin2z_final)))
    h1_aligned, h2_aligned = coalign_waveforms(hp_target, TimeSeries(hp_test, delta_t=delta_T))

    # Plot of the coaligned waveforms
    plt.figure(figsize=(12, 5))
    plt.plot(h1_aligned.sample_times, h1_aligned, label = f'Target')
    plt.plot(h2_aligned.sample_times, h2_aligned, label = f'Template.', linestyle='dashed')
    plt.title(f'The mismatch between the gravitational waves is {1-max_match}')
    plt.xlabel('Time (s)')
    plt.ylabel('Strain')
    plt.legend()

    plt.savefig('./Graphics/main2.png', bbox_inches='tight') # Guardar la imagen como un png
    #plt.show() # Mostrar la imagen


    return max_match, time.time()-time_initial, "None", prms_final

# Extrinsic before LongAscNodes and Polarization

In [ ]:
def main_Full(): 

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    n_workers = 6

    # We write the initial values for the optimization

    mass1_template = [8*lal.MSUN_SI, 8*lal.MSUN_SI, 100*lal.MSUN_SI, 100*lal.MSUN_SI, 40*lal.MSUN_SI, 40*lal.MSUN_SI]
    mass2_template = [4*lal.MSUN_SI, 4*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI]
    eff_spin_template = [-0.5, 0.5, 0.7, -0.7, 0.7, -0,7]


    prms_initial = []
    for i in range(n_workers):
        mass_ratio_template = mass1_template[i]/mass2_template[i]
        chirp_mass_template = chirp_mass_function([mass1_template[i], mass2_template[i]])
        prms_initial.append([mass_ratio_template, chirp_mass_template, eff_spin_template[i]])

    print("Starting Hierarchical Optimization")
    print("First optimization: Masses and Effective Spin")
    time_initial=time.time()

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_first)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")
    
    # max_match, prms_final = opt_first(prms_initial[3])
    
    print(f"The match of the first optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(f"The first optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2]}.") # Print the parameters calculated by the global optimization

    print(f"Second Optimization: Spin2z and Inclination")

    spin2z_template = [0.0, 0.0 ,-0.4, 0.4, -0.7, 0.7]
    incl_template = [0, math.pi/2 ,math.pi/2, math.pi/2, 3*math.pi/4, 3*math.pi/4]

    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], spin2z_template[i], incl_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_second_incl)(prms) for prms in prms_initial)

    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")


    print(f"The match of the second optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(f"The second optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2], prms_final[3]}.") # Print the parameters calculated by the global optimization
    print(f"inclination: {prms_final[4]}, incl_target: {incl_target}")

    print(f"Third Optimization: Spin1x and Spin1y")


    spin1x_template = [0.0, 0.4, -0.4, -0.4, 0.4, 0]
    spin1y_template = [0.0, 0.4, -0.4, 0.4, -0.4, 0]
    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], prms_final[3], spin1x_template[i], spin1y_template[i], prms_final[4]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_third_precession)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")

    # max_match, prms_final = opt_third_precession(prms_initial[2])

    masses_final = func.M_c_and_q_m(prms_final[0], prms_final[1])
    spin1z_final, spin2z_final = func.Eff_spin_and_spin1(masses_final[0], masses_final[1], prms_final[2], prms_final[3])

    Comp_time = time.time()-time_initial

    print(f"The match of the total optimization is: {max_match}. The complete optimization took {Comp_time} seconds.")
    
    print(f"Optimum at masses {masses_final[0]/lal.MSUN_SI} and {masses_final[1]/lal.MSUN_SI} solar masses and spins {prms_final[4], prms_final[5], spin1z_final} and {spin2z_final}")
    print(f"The target: {parameters_target.m1/lal.MSUN_SI} and {parameters_target.m2/lal.MSUN_SI} solar masses and spins {parameters_target.s1x, parameters_target.s1y, parameters_target.s1z} and {parameters_target.s2z}")
    print(f"chirp mass: {prms_final[1]/lal.MSUN_SI}, chirp mass target: {parameters_target.chirp_mass()/lal.MSUN_SI}")
    print(f"effective spin parameter: {prms_final[2]}, eff_spin_target: {parameters_target.eff_spin()}")
    print(f"inclination: {prms_final[6]}, incl_target: {incl_target}")

    # Waveforms Coaligned 
    hp_test, hc_test, time_test = simulationTD(params(masses_final, (prms_final[4],prms_final[5],spin1z_final), (0,0,spin2z_final)))
    h1_aligned, h2_aligned = coalign_waveforms(hp_target, TimeSeries(hp_test, delta_t=delta_T))

    # Plot of the coaligned waveforms
    plt.figure(figsize=(12, 5))
    plt.plot(h1_aligned.sample_times, h1_aligned, label = f'Target')
    plt.plot(h2_aligned.sample_times, h2_aligned, label = f'Template.', linestyle='dashed')
    plt.title(f'The mismatch between the gravitational waves is {1-max_match}')
    plt.xlabel('Time (s)')
    plt.ylabel('Strain')
    plt.legend()

    plt.savefig('./Graphics/main2.png', bbox_inches='tight') # Guardar la imagen como un png
    plt.show() # Mostrar la imagen


    return max_match, Comp_time, prms_final

# One Body Precession. Optimizing Chi_x and Chi_y. Both Extrinsic and Intrinsic

In [ ]:
def opt_third_precession_not_incl(prms_initial:list)->tuple:

    opt = nlopt.opt(nlopt.LN_NELDERMEAD, 6) # Define the local optimization
    opt.set_lower_bounds([1, lal.MSUN_SI, -1, -1, -1, -1]) # Same boundaries as before
    opt.set_upper_bounds([20, 175*lal.MSUN_SI, 1, 1, 1, 1]) # Same boundaries as before
    opt.set_min_objective(func.opt_match_precession_non_incl)
    opt.set_ftol_rel(-1+1e-4) # Tolerance used for tests
    #opt.set_xtol_rel(1e-4) # Tolerance used for computations

    prms_final = opt.optimize(prms_initial) # We use the parameters obtained by the global optimization as the starting point
    max_match = -opt.last_optimum_value()

    print(colored(f"Number of Evaluations: {opt.get_numevals()} made by initial conditions: {prms_initial[4], prms_initial[5]}", "cyan"))

    return max_match, prms_final


def opt_third_precession(prms_initial:list)->tuple:

    opt = nlopt.opt(nlopt.LN_NELDERMEAD, 8) # Define the local optimization
    opt.set_lower_bounds([1, lal.MSUN_SI, -1, -1, -1, -1, 0, 0]) # Same boundaries as before
    opt.set_upper_bounds([20, 175*lal.MSUN_SI, 1, 1, 1, 1, 2*math.pi, math.pi]) # Same boundaries as before
    opt.set_min_objective(func.opt_match_precession)
    #opt.set_ftol_rel(-1+1e-4) # Tolerance used for tests
    opt.set_xtol_rel(1e-4) # Tolerance used for computations

    prms_final = opt.optimize(prms_initial) # We use the parameters obtained by the global optimization as the starting point
    max_match = -opt.last_optimum_value()

    print(colored(f"Number of Evaluations: {opt.get_numevals()} made by initial conditions: {prms_initial[4], prms_initial[5]}", "cyan"))

    return max_match, prms_final


def main_not_extrinsic(): 

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    n_workers = 6

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    mass1_template = [8*lal.MSUN_SI, 8*lal.MSUN_SI, 100*lal.MSUN_SI, 100*lal.MSUN_SI, 40*lal.MSUN_SI, 40*lal.MSUN_SI]
    mass2_template = [4*lal.MSUN_SI, 4*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI]
    eff_spin_template = [-0.5, 0.5, 0.7, -0.7, 0.7, -0,7]

    prms_initial = []
    for i in range(n_workers):
        mass_ratio_template = mass1_template[i]/mass2_template[i]
        chirp_mass_template = chirp_mass_function([mass1_template[i], mass2_template[i]])
        prms_initial.append([mass_ratio_template, chirp_mass_template, eff_spin_template[i]])

    print("Starting Hierarchical Optimization")
    print("First optimization: Masses and Effective Spin")
    time_initial=time.time()

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_first)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")
    
    # max_match, prms_final = opt_original_first(prms_initial[3])
    
    print(f"The match of the first optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(f"The first optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2]}.") # Print the parameters calculated by the global optimization

    print(f"Second Optimization: Spin2z and Relevant Precession Parameters")

    n_workers = 5
        
    spin2z_template = [0.0, -0.7, 0.7, -0.7, 0.7]
    spin1x_template = [0.4, 0.0, 0.0, -0.4, -0.4]
    spin1y_template = [0.4, 0.0, 0.0, 0.4, 0.4]
    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], spin2z_template[i], spin1x_template[i], spin1y_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_third_precession_not_incl)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")

    # max_match, prms_final = opt_precession_high_Q(prms_initial[2])

    masses_final = func.M_c_and_q_m(prms_final[0], prms_final[1])
    spin1z_final, spin2z_final = func.Eff_spin_and_spin1(masses_final[0], masses_final[1], prms_final[2], prms_final[3])

    Comp_time = time.time()-time_initial

    print(colored(f"The match of the total optimization is: {max_match}. The complete optimization took {Comp_time} seconds.", "magenta"))
    
    print(f"Parameters: {prms_final}")
    print(f"chirp mass: {prms_final[1]/lal.MSUN_SI}, chirp mass target: {parameters_target.chirp_mass()/lal.MSUN_SI}")
    print(f"effective spin parameter: {prms_final[2]}, eff_spin_target: {parameters_target.eff_spin()}")
    
    # Waveforms Coaligned 
    hp_test, hc_test, time_test = simulationTD(params(masses_final, (prms_final[4],prms_final[5],spin1z_final), (0,0,spin2z_final)))
    h1_aligned, h2_aligned = coalign_waveforms(hp_target, TimeSeries(hp_test, delta_t=delta_T))

    # Plot of the coaligned waveforms
    plt.figure(figsize=(12, 5))
    plt.plot(h1_aligned.sample_times, h1_aligned, label = f'Target')
    plt.plot(h2_aligned.sample_times, h2_aligned, label = f'Template.', linestyle='dashed')
    plt.title(f'The mismatch between the gravitational waves is {1-max_match}')
    plt.xlabel('Time (s)')
    plt.ylabel('Strain')
    plt.legend()

    plt.savefig('./Graphics/main2.png', bbox_inches='tight') # Guardar la imagen como un png
    plt.show() # Mostrar la imagen


    return max_match, Comp_time, prms_final





def main_Full(): 

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    n_workers = 6

    # We write the initial values for the optimization

    mass1_template = [8*lal.MSUN_SI, 8*lal.MSUN_SI, 100*lal.MSUN_SI, 100*lal.MSUN_SI, 40*lal.MSUN_SI, 40*lal.MSUN_SI]
    mass2_template = [4*lal.MSUN_SI, 4*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI]
    eff_spin_template = [-0.5, 0.5, 0.7, -0.7, 0.7, -0,7]


    prms_initial = []
    for i in range(n_workers):
        mass_ratio_template = mass1_template[i]/mass2_template[i]
        chirp_mass_template = chirp_mass_function([mass1_template[i], mass2_template[i]])
        prms_initial.append([mass_ratio_template, chirp_mass_template, eff_spin_template[i]])

    print("Starting Hierarchical Optimization")
    print("First optimization: Masses and Effective Spin")
    time_initial=time.time()

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_first)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")
    
    # max_match, prms_final = opt_first(prms_initial[3])
    
    print(f"The match of the first optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(f"The first optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2]}.") # Print the parameters calculated by the global optimization

    print(f"Second Optimization: Spin2z and Inclination")

    spin2z_template = [0.0, 0.0 ,-0.4, 0.4, -0.7, 0.7]
    incl_template = [0, math.pi/2 ,math.pi/2, math.pi/2, 3*math.pi/2, 3*math.pi/2]

    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], spin2z_template[i], incl_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_second_incl)(prms) for prms in prms_initial)

    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")


    print(f"The match of the second optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(f"The second optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2], prms_final[3]}.") # Print the parameters calculated by the global optimization
    print(f"inclination: {prms_final[4]}, incl_target: {incl_target}")

    print(f"Third Optimization: Spin1x and Spin1y")


    spin1x_template = [0.0, 0.0,0.4, -0.4, -0.4, 0.4]
    spin1y_template = [0.0, 0.0, 0.4, -0.4, 0.4, -0.4]
    LongAscNodes_template = [0, math.pi/4, 0, 0, 0, 0]

    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], prms_final[3],
                              spin1x_template[i], spin1y_template[i], prms_final[4], LongAscNodes_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_third_precession)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")

    # max_match, prms_final = opt_third_precession(prms_initial[2])

    masses_final = func.M_c_and_q_m(prms_final[0], prms_final[1])
    spin1z_final, spin2z_final = func.Eff_spin_and_spin1(masses_final[0], masses_final[1], prms_final[2], prms_final[3])

    Comp_time = time.time()-time_initial

    print(f"The match of the total optimization is: {max_match}. The complete optimization took {Comp_time} seconds.")
    
    print(f"Optimum at masses {masses_final[0]/lal.MSUN_SI} and {masses_final[1]/lal.MSUN_SI} solar masses and spins {prms_final[4], prms_final[5], spin1z_final} and {spin2z_final}")
    print(f"The target: {parameters_target.m1/lal.MSUN_SI} and {parameters_target.m2/lal.MSUN_SI} solar masses and spins {parameters_target.s1x, parameters_target.s1y, parameters_target.s1z} and {parameters_target.s2z}")
    print(f"chirp mass: {prms_final[1]/lal.MSUN_SI}, chirp mass target: {parameters_target.chirp_mass()/lal.MSUN_SI}")
    print(f"effective spin parameter: {prms_final[2]}, eff_spin_target: {parameters_target.eff_spin()}")
    print(f"inclination: {prms_final[6]}, incl_target: {incl_target}")
    print(f"longascnodes: {prms_final[7]}, longascnodes_target: {LongAscNodes_target}")

    # Waveforms Coaligned 

    optimized_params = params(masses_final, (prms_final[4],prms_final[5],spin1z_final), (0,0,spin2z_final),
                                                       incl = prms_final[6], longAscNodes = prms_final[7])

    hp_test, hc_test, time_test = simulationTD(optimized_params)
    h1_aligned, h2_aligned = coalign_waveforms(hp_target, TimeSeries(hp_test, delta_t=delta_T))

    # Plot of the coaligned waveforms
    plt.figure(figsize=(12, 5))
    plt.plot(h1_aligned.sample_times, h1_aligned, label = f'Target')
    plt.plot(h2_aligned.sample_times, h2_aligned, label = f'Template.', linestyle='dashed')
    plt.title(f'The mismatch between the gravitational waves is {1-max_match}')
    plt.xlabel('Time (s)')
    plt.ylabel('Strain')
    plt.legend()

    plt.savefig('./Graphics/main2.png', bbox_inches='tight') # Guardar la imagen como un png
    plt.show() # Mostrar la imagen


    return max_match, Comp_time, optimized_params

# PRECOMPARISON WITH BALEARES CODE

In [ ]:
def main_Full(): 

    # We write the initial values for the optimization (it is not important if the first optimization is global)
    n_workers = 6

    # We write the initial values for the optimization

    mass1_template = [8*lal.MSUN_SI, 8*lal.MSUN_SI, 100*lal.MSUN_SI, 100*lal.MSUN_SI, 40*lal.MSUN_SI, 40*lal.MSUN_SI]
    mass2_template = [4*lal.MSUN_SI, 4*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI, 10*lal.MSUN_SI]
    eff_spin_template = [-0.5, 0.5, 0.7, -0.7, 0.7, -0,7]


    prms_initial = []
    for i in range(n_workers):
        mass_ratio_template = mass1_template[i]/mass2_template[i]
        chirp_mass_template = chirp_mass_function([mass1_template[i], mass2_template[i]])
        prms_initial.append([mass_ratio_template, chirp_mass_template, eff_spin_template[i]])

    print("Starting Hierarchical Optimization")
    print("First optimization: Masses and Effective Spin")
    time_initial=time.time()

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_first)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")
    
    # max_match, prms_final = opt_first(prms_initial[3])
    
    print(f"The match of the first optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(f"The first optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2]}.") # Print the parameters calculated by the global optimization

    print(f"Second Optimization: Spin2z, Chi_p and Inclination")

    spin2z_template = [0.0, 0.0 ,-0.4, 0.4, -0.7, 0.7]
    chi_p_template = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
    incl_template = [0, math.pi/2 ,math.pi/2, math.pi/2, 3*math.pi/2, 3*math.pi/2]

    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], spin2z_template[i], chi_p_template[i], incl_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_second_incl)(prms) for prms in prms_initial)

    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")


    print(f"The match of the second optimization is: {max_match}. It took {time.time()-time_initial} seconds.")
    print(f"The second optimization has obtain the following parameters: {prms_final[0], prms_final[1]/lal.MSUN_SI, prms_final[2], prms_final[3], prms_final[4]}.") # Print the parameters calculated by the global optimization
    print(f"inclination: {prms_final[4]}, incl_target: {incl_target}")

    print(f"Third Optimization: Spin1x and Spin1y")


    anglespin1_template = [-math.pi/2, -math.pi/2, -math.pi/2, math.pi/2, math.pi/2, math.pi/2]
    LongAscNodes_template = [0, math.pi/4, 0, math.pi/4, 0, math.pi/4]
    pol_template = [0, 0, math.pi/4, math.pi/4, math.pi/4, 0] #Polarization of the GW (Periodic [0,pi/2])

    prms_initial = []
    for i in range(n_workers):
        prms_initial.append([prms_final[0], prms_final[1], prms_final[2], prms_final[3],
                              prms_final[4], anglespin1_template[i], prms_final[5], LongAscNodes_template[i], pol_template[i]])

    results_multiprocess = Parallel(n_jobs=n_workers)(delayed(opt_third_precession)(prms) for prms in prms_initial)
    
    max_match=0 
    for i in range(n_workers):
        if (results_multiprocess[i][0])>max_match:
            max_match = results_multiprocess[i][0]
            prms_final = results_multiprocess[i][1]
            best_worker = i 
    print(f"The best initial parameters were {prms_initial[best_worker]}")

    # max_match, prms_final = opt_third_precession(prms_initial[2])

    masses_final = func.M_c_and_q_m(prms_final[0], prms_final[1])
    spin1z_final, spin2z_final = func.Eff_spin_and_spin1(masses_final[0], masses_final[1], prms_final[2], prms_final[3])
    spin1x_final, spin1y_final = func.spin1perp_and_angle(prms_final[4], prms_final[5])
    pol_final = prms_final[8]

    Comp_time = time.time()-time_initial

    print(f"The match of the total optimization is: {max_match}. The complete optimization took {Comp_time} seconds.")
    
    print(f"Optimum at masses {masses_final[0]/lal.MSUN_SI} and {masses_final[1]/lal.MSUN_SI} solar masses and spins {spin1x_final, spin1y_final, spin1z_final} and {spin2z_final}")
    print(f"The target: {parameters_target.m1/lal.MSUN_SI} and {parameters_target.m2/lal.MSUN_SI} solar masses and spins {parameters_target.s1x, parameters_target.s1y, parameters_target.s1z} and {parameters_target.s2z}")
    print(f"chirp mass: {prms_final[1]/lal.MSUN_SI}, chirp mass target: {parameters_target.chirp_mass()/lal.MSUN_SI}")
    print(f"effective spin parameter: {prms_final[2]}, eff_spin_target: {parameters_target.eff_spin()}")
    print(f"inclination: {prms_final[6]}, incl_target: {incl_target}")
    print(f"longascnodes: {prms_final[7]}, longascnodes_target: {LongAscNodes_target}")

    # Waveforms Coaligned 

    optimized_params = params(masses_final, (spin1x_final, spin1y_final, spin1z_final), (0,0,spin2z_final),
                                                       incl = prms_final[6], longAscNodes = prms_final[7])

    hp_test, hc_test, time_test = simulationTD(optimized_params)
    hp_test, hc_test = TimeSeries(hp_test, delta_t=delta_T), TimeSeries(hc_test, delta_t=delta_T)
    h_test = hp_test*math.cos(2*pol_final)+hc_test*math.sin(2*pol_final)

    h1_aligned, h2_aligned = coalign_waveforms(h_target, h_test)

    # Plot of the coaligned waveforms
    plt.figure(figsize=(12, 5))
    plt.plot(h1_aligned.sample_times, h1_aligned, label = f'Target')
    plt.plot(h2_aligned.sample_times, h2_aligned, label = f'Template.', linestyle='dashed')
    plt.title(f'The mismatch between the gravitational waves is {1-max_match}')
    plt.xlabel('Time (s)')
    plt.ylabel('Strain')
    plt.legend()

    plt.savefig('./Graphics/main2.png', bbox_inches='tight') # Guardar la imagen como un png
    plt.show() # Mostrar la imagen

    return max_match, Comp_time, optimized_params, pol_final


"""To make the comparison with Eleanor's code more fair we will not use multiprocessing. Moreover we optimize over LongAscNodes 
instead of PhiRef"""